# Create a CPW meander Geometry

In [18]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
from qiskit_metal import draw, Dict, designs, MetalGUI
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.core import QRoute, QRoutePoint
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround

In [50]:
import math
import numpy as np
import matplotlib.pyplot as plt
from forge.meander import Meander

In [5]:
def get_parameters(w, s, Gn, L, h):
    """
    Returns x, n, device width, and area
    """
    n=1
    W = w+2*s
    r = (Gn+1)/2 * W
    l1 = h-2*r-W
    l2 = l1/2 - r - 0.5*W
    if l1<0 or l2<0:
        raise ValueError
    while(True):
        _n = n+1
        _val = (_n+1)*np.pi*r + (_n-1)*l1 + 2*l2
        _x = 0.5*(L-_val)
        if _x < 0:
            if _n == 2:
                raise ValueError("Check Dimensions")
            else:
                break
        n += 1
        
    val = (n+1)*np.pi*r + (n-1)*l1 + 2*l2
    x = 0.5*(L-val)
    
    if x<0:
        raise AttributeError
        
    W = 2*x+ (2*n+2)*r
    
    return W, n, x

In [17]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'
hfss = design.renderers.hfss

gui = MetalGUI(design)
design.delete_all_components()

design.overwrite_enabled = True
design.delete_all_components()

In [47]:
class BendedEnds(QComponent):
    component_metadata = Dict(
        short_name='BendedEnds',
        _qgeometry_table_path='True'
    )
    default_options = Dict(
            origin_x = "0um",
            origin_y = "0um",
            x_length = "0.2mm",
            trace_gap = "0.01mm",
            trace_width = "0.01mm",
            meander_radius = "0.5mm",
            n_points_meander = 128,
            orientation = True,
            straight_first = True, 
            chip="main",
            layer="1"
    )
    
    TOOLTIP = "Half radius and straigth edge on end"
    
    def make(self):
        
        p = self.p
        origin_x, origin_y = p.origin_x, p.origin_y
        n_points_meander = p.n_points_meander
        theta = np.pi/2
        d_theta = theta/n_points_meander
        orientation = p.orientation
        straight_first = p.straight_first
        meander_radius = p .meander_radius
        angles = np.arange(-theta, 0, d_theta)
        xs = meander_radius * np.cos(angles)
        if orientation:
            ys = -meander_radius * np.sin(angles) - meander_radius
        else:
            ys = meander_radius * np.sin(angles) + meander_radius
        
        
        if p.straight_first:
            x_pts, y_pts = [np.array([origin_x])], [np.array([origin_y])]
            x, y = origin_x + p.x_length, origin_y
            x_pts.append(xs+x)
            y_pts.append(ys+y)
        else:
            x_pts, y_pts = [xs+origin_x], [ys+origin_y]

            x, y = xs[-1] + origin_x + p.x_length, ys[-1] + origin_y
            x_pts.append(np.array([x]))
            y_pts.append(np.array([y]))
        
        x_pts = np.concatenate(x_pts)
        y_pts = np.concatenate(y_pts)
        self.points = np.array([x_pts, y_pts]).T
        self.qgeometry_table_usage = {'path': True, 'poly': False, 'junction': False}
        self.make_elements(self.points)
        self.x_end = x_pts[-1]
        self.y_end = y_pts[-1]
        self.x_pts = x_pts
        self.y_pts = y_pts
        
        
    def make_elements(self, pts: np.ndarray):
        """Turns the CPW points into design elements, and add them to the
        design object.

        Args:
            pts (np.ndarray): Array of points
        """
        p = self.p
        # prepare the routing track
        line = draw.LineString(pts)

        # expand the routing track to form the substrate core of the cpw
        self.add_qgeometry(
                'path', {'trace': line},
                width=p.trace_width,
                layer=p.layer,
                fillet=0
                )
        self.add_qgeometry('path', {'cut': line},
                               width=p.trace_width + 2 * p.trace_gap,
                               layer=p.layer,
                               subtract=True,
                               fillet=0
                               )
        
    def get_endpoints(self):
        return self.x_end, self.y_end
    
    
    def get_points(self):
        return self.x_pts, self.y_pts

In [83]:
design.delete_all_components()
b1 = BendedEnds(
    design=design, 
    name="bend1", 
    options= Dict(
            hfss_wire_bonds = True,
            origin_x = "0um",
            origin_y = "0um",
            x_length = "0.1mm",
            n_points_meander = "128", 
            trace_gap = "0.02mm",
            trace_width = "0.02mm",
            meander_radius = "0.5mm",
            orientation = True,
            straight_first = True, 
            chip="main",
            layer="1"
    )
)
origin_x, origin_y = b1.get_endpoints()
m = Meander(
    design=design, 
    name="meanders", 
    options= Dict(
            hfss_wire_bonds = True,
            origin_x = origin_x,
            origin_y = origin_y,
            n_points_meander = 128, 
            width = "9mm",
            width_incl_meanders = "false",
            spacing = "1mm",
            n_lines = 5,
            meander_radius = "0.5mm",
            coupler_length = "4mm",
            coupler_spacing = "1mm",
            coupler_radius = "0.5mm",
            n_points_coupler = 256,
            coupler_orientation = "false",
            trace_gap = "0.02mm",
            trace_width = "0.02mm",
            end_length = "9mm",
            pin_width = "0.01mm",
            type  = "CPW",
            layer = "1",
            chip = "main",
            fillet = 0,
    )
)
gui.rebuild()
gui.autoscale()